In [65]:
import os
from openslide import OpenSlide
import cv2
import xml.etree.ElementTree as ET
import numpy as np
import pandas as pd
import openslide
from PIL import Image
import math


In [66]:
src = '/home/kuki2070s2/PycharmProjects/TeamKyu/Michael/5619_Wirtz.Denis_OTS-19_5619-009.svs'
#svss = [_ for _ in os.listdir(src) if _.endswith('svs')]
#svs = svss[0]
svs = src
print(type(src))


#svs = OpenSlide(os.path.join(src,svs))

# xmls = [_ for _ in os.listdir(src) if _.endswith('xml')]
xml = '/home/kuki2070s2/PycharmProjects/TeamKyu/Michael/5619_Wirtz.Denis_OTS-19_5619-009.xml'

tree = ET.parse(os.path.join(src,xml))
root = tree.getroot()
# pre-allocate arrays of interest
x = np.array([])
y = np.array([])
obj = np.array([])
label = np.array([])

for Annotation in root.iter('Annotation'):
  for Region in Annotation.iter('Region'):
     xx = np.array([int(Vertex.get('X')) for Vertex in Region.iter('Vertex')])
     yy = np.array([int(Vertex.get('Y')) for Vertex in Region.iter('Vertex')])
     objj = np.array([int(Region.get('Id'))]*len(xx))
     labell = np.array([int(Annotation.get('Id'))]*len(xx))
     x = np.concatenate((x,xx),axis=None)
     y = np.concatenate((y,yy),axis=None)
     obj = np.concatenate((obj,objj),axis=None)
     label = np.concatenate((label,labell),axis=None)
print('number of coordinates in annotation : ',len(x))
mdict = {'x':x,'y':y,'objID':obj,'label':label}
df = pd.DataFrame(mdict)

<class 'str'>
number of coordinates in annotation :  50139


In [67]:
for classidx, classid in enumerate(np.unique(df['label'])[0:1]):
    for objidx, objid in enumerate(np.unique(df['objID'])[0:1]):
        #if np.isnan(classid) and np.isnan(classid) != True:
            object = df[df['label']==classid][df['objID']==objid]
            x = object['x']
            y = object['y']
            xmin = np.min(x)
            xmax = np.max(x)
            ymin = np.min(y)
            ymax = np.max(y)
            width = xmax-xmin
            height = ymax - ymin
            if np.isnan(width): continue
            size = (int(width),int(height))
            img= openslide.OpenSlide(svs)
            region = img.read_region(location=(int(xmin),int(ymin)),level=0,size=size)
            vert = np.array([[i-xmin,j-ymin] for i,j in zip(x,y)]).astype(np.int32)
            mask = np.zeros(size)
            cv2.fillPoly(mask,[vert],color=255)
            mask = mask.astype(np.uint16)
            #mask.save('class'+str(classidx)+'obj'+str(objidx)+'.tif')

            #math = Image.fromarray(mask)
            mask_dst = '/home/kuki2070s2/PycharmProjects/TeamKyu/Sam/image_dst_tiles'
            mask_fn = 'class'+str(classidx)+'obj'+str(objidx)+'.tif'
            #mask.save(os.path.join(mask_dst,mask_fn))
            cv2.imwrite(('class'+str(classidx)+'obj'+str(objidx)+'.tif'), mask)
            region_fn = 'class'+str(classidx)+'obj'+str(objidx)+'.jpg'
            region.convert('RGB').save(os.path.join(mask_dst,region_fn))

/home/kuki2070s2/anaconda3/envs/TF2Shared/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


In [68]:
            #tiling
            if np.min(mask.size) < 256:
                mask_min = np.min(mask.size)
                factor = 256/mask_min
                #for x in mask.size:
                    #mask.size = int(round(x*factor))
                mask.size = [int(round(factor*x)) for x in mask.size]
            tiledim = (256,256)
            offset = (256,256)
            mask_shape = mask.shape

            for i in range(int(math.ceil(mask_shape[0]/(offset[1]*1.0)))):
                for j in range(int(math.ceil(mask_shape[1]/(offset[0]*1.0)))):
                   #img[offset[1]*i:min(offset[1]*i+tile_size[1], img_shape[0]), offset[0]*j:min(offset[0]*j+tile_size[0], img_shape[1])]
                    crop = img[offset[1]*i:min(offset[1]*i+tiledim[1],mask_shape[0]), offset[0]*j:min(offset[0]*j+tiledim[0],mask_shape[1])]
                    #cv2.imwrite("debug"+"_"+str(j)+".png",crop)

                    crop_dst = '/home/kuki2070s2/PycharmProjects/TeamKyu/Sam/image_dst_tiles'
                    crop_fn = "debug"+"_"+str(j)+".png"
                    crop.save(os.path.join(crop_dst,crop_fn))



TypeError: 'OpenSlide' object is not subscriptable